In [1]:
# Standard library
import itertools
from collections import defaultdict

# Third-party libraries
import numpy as np
import pandas as pd
import scanpy as sc

# Local package: cna_inferer
from cna_inferer.main import process_and_call_cnas


In [2]:
adata_r1 = process_and_call_cnas("GSM3814888_day8_rep1_filtered_gene_bc_matrices_h5.h5")
adata_r2 = process_and_call_cnas("GSM3814889_day8_rep2_filtered_gene_bc_matrices_h5.h5")
adata_rc11 = process_and_call_cnas("GSM3814894_rc11_day8_filtered_gene_bc_matrices_h5.h5")

c:\Users\12039\anaconda3\Lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'ccds_id', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'protein_id', 'protein_version']
c:\Users\12039\anaconda3\Lib\site-packages\scanpy\preprocessing\_simple.py:287: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number


✅ Processed 0 cells...
✅ Processed 500 cells...
✅ Processed 1000 cells...
✅ Processed 1500 cells...
✅ Processed 2000 cells...
✅ Processed 2500 cells...
✅ Processed 3000 cells...
✅ Processed 3500 cells...
✅ Processed 4000 cells...
📊 Total CNA events in GSM3814888_day8_rep1_filtered_gene_bc_matrices_h5.h5: 1229


c:\Users\12039\anaconda3\Lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'ccds_id', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'protein_id', 'protein_version']
c:\Users\12039\anaconda3\Lib\site-packages\scanpy\preprocessing\_simple.py:287: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number


✅ Processed 0 cells...
✅ Processed 500 cells...
✅ Processed 1000 cells...
✅ Processed 1500 cells...
✅ Processed 2000 cells...
📊 Total CNA events in GSM3814889_day8_rep2_filtered_gene_bc_matrices_h5.h5: 18


c:\Users\12039\anaconda3\Lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'ccds_id', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'protein_id', 'protein_version']
c:\Users\12039\anaconda3\Lib\site-packages\scanpy\preprocessing\_simple.py:287: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number


✅ Processed 0 cells...
✅ Processed 500 cells...
✅ Processed 1000 cells...
✅ Processed 1500 cells...
✅ Processed 2000 cells...
✅ Processed 2500 cells...
✅ Processed 3000 cells...
✅ Processed 3500 cells...
✅ Processed 4000 cells...
✅ Processed 4500 cells...
✅ Processed 5000 cells...
✅ Processed 5500 cells...
✅ Processed 6000 cells...
✅ Processed 6500 cells...
✅ Processed 7000 cells...
✅ Processed 7500 cells...
📊 Total CNA events in GSM3814894_rc11_day8_filtered_gene_bc_matrices_h5.h5: 287


In [3]:


# Sample dictionary
samples = {
    "rep1": adata_r1,
    "rep2": adata_r2,
    "rc11": adata_rc11
}

# Gene → bin mapping (consistent with earlier usage)
w_size = 150
n_bins = adata_r1.obsm["X_binned"].shape[1]
gene_bins = pd.Series(
    np.arange(adata_r1.n_vars) // w_size,
    index=adata_r1.var_names
)

# Count in how many cells each gene appears in a CNA region
def get_gene_cell_counts(adata):
    gene2cells = defaultdict(set)
    for cell, calls in zip(adata.obs_names, adata.obs["cna_calls"]):
        for ev in calls:
            # Support both start_bin/end_bin and start/end keys
            b0 = ev.get("start_bin", ev.get("start"))
            b1 = ev.get("end_bin",   ev.get("end"))
            genes = gene_bins[(gene_bins >= b0) & (gene_bins <= b1)].index
            for g in genes:
                gene2cells[g].add(cell)
    return gene2cells

# Filtering threshold: must appear in at least 100 cells or ≥ 2% of cells
min_cells = 100
pct_of_cells = .02

# Build filtered gene sets for each sample
sample_filtered_genes = {}
for name, ad in samples.items():
    gene2cells = get_gene_cell_counts(ad)
    n_cells = ad.n_obs
    filtered = {
        g for g, cells in gene2cells.items()
        if (len(cells) >= min_cells and len(cells)/n_cells >= pct_of_cells)
    }
    sample_filtered_genes[name] = filtered

# Extract individual sets
rep1 = sample_filtered_genes["rep1"]
rep2 = sample_filtered_genes["rep2"]
rc11 = sample_filtered_genes["rc11"]

# 1. Union of all three
union_all = rep1 | rep2 | rc11
print(f"Union of all genes ({len(union_all)}):")
print(sorted(union_all) if union_all else "No genes in union", "\n")

# 2. Common to all three
common_all = rep1 & rep2 & rc11
print(f"Genes common to all samples ({len(common_all)}):")
print(sorted(common_all) if common_all else "No genes in common", "\n")

# 3. Pairwise intersections
for a, b in itertools.combinations(samples.keys(), 2):
    inter = sample_filtered_genes[a] & sample_filtered_genes[b]
    print(f"Intersection of {a} & {b} ({len(inter)}):")
    print(sorted(inter) if inter else "No overlapping genes", "\n")


Union of all genes (1500):
['ABCC4', 'ABHD13', 'ABHD4', 'ABTB2', 'AC000403.4', 'AC001226.7', 'AC015691.13', 'AC090587.5', 'ACCS', 'ACIN1', 'ACP2', 'ACTN1', 'ACTR10', 'ADAM8', 'ADCY4', 'ADM', 'ADPRHL1', 'ADRBK1', 'AGBL2', 'AHNAK', 'AIP', 'AJUBA', 'AKAP11', 'AKAP5', 'AKAP6', 'AKIP1', 'AL161668.5', 'AL356585.2', 'ALDH3B1', 'ALG11', 'ALG5', 'ALKBH3', 'ALOX5AP', 'AMBRA1', 'AMER2', 'AMPD3', 'ANAPC15', 'ANG', 'ANKRD10', 'ANKRD13D', 'ANO1', 'ANO1-AS2', 'ANO5', 'AP000438.2', 'AP000442.1', 'AP000442.4', 'AP000487.5', 'AP000640.10', 'AP000769.7', 'AP001257.1', 'AP001258.4', 'AP001462.6', 'AP003068.23', 'AP003068.9', 'AP003419.16', 'AP003774.6', 'AP006285.6', 'AP006621.5', 'AP1G2', 'AP2A2', 'AP4S1', 'AP5B1', 'AP5M1', 'APBB1', 'APEX1', 'API5', 'APIP', 'APLNR', 'ARAP1', 'ARF6', 'ARFGAP2', 'ARFIP2', 'ARG2', 'ARGLU1', 'ARHGAP1', 'ARHGAP5', 'ARHGAP5-AS1', 'ARHGEF17', 'ARHGEF40', 'ARHGEF7', 'ARID4A', 'ARL11', 'ARL14EP', 'ARL2', 'ARNTL', 'ART5', 'ASCL2', 'ASRGL1', 'ATG13', 'ATG14', 'ATG16L2', 'ATG2A', 'A

In [4]:
# Assume adata_r1, adata_r2, adata_rc11 are already defined
samples = {
    "rep1": adata_r1,
    "rep2": adata_r2,
    "rc11": adata_rc11
}

# Parameters
window_size   = 150
min_cells     = 100
pct_of_cells  = 0.02
bin_window    = 10_000_000  # 10Mb

# 1. Build gene → cell mapping
def get_gene_cell_counts(adata):
    gene2cells = defaultdict(set)
    # Construct gene → bin mapping
    gene_bins = pd.Series(
        np.arange(adata.n_vars) // window_size,
        index=adata.var_names
    )
    for cell, calls in zip(adata.obs_names, adata.obs["cna_calls"]):
        for ev in calls:
            b0 = ev.get("start_bin", ev.get("start"))
            b1 = ev.get("end_bin",   ev.get("end"))
            genes = gene_bins[gene_bins.between(b0, b1)].index
            for g in genes:
                gene2cells[g].add(cell)
    return gene2cells

# 2. Filter each sample
sample_filtered = {}
for name, ad in samples.items():
    gene2cells = get_gene_cell_counts(ad)
    nc = ad.n_obs
    kept = {
        g for g, cells in gene2cells.items()
        if len(cells) >= min_cells and len(cells) / nc >= pct_of_cells
    }
    sample_filtered[name] = kept

# 3. Get shared genes between rep1 and rc11
shared = sample_filtered["rep1"] & sample_filtered["rc11"]

# 4. Extract gene annotation
df_anno = adata_r1.var.loc[sorted(shared), ["chromosome", "start", "end"]].copy()

# 5. Compute 10Mb genomic bins
df_anno["bin_start_10Mb"] = (df_anno["start"] // bin_window) * bin_window

# 6. Count gene frequency per chromosome
chr_counts = df_anno["chromosome"].value_counts().reset_index()
chr_counts.columns = ["chromosome", "gene_count"]

# 7. Count genes per 10Mb bin and get top 10 enriched regions
hotspots = (
    df_anno
    .groupby(["chromosome", "bin_start_10Mb"])
    .size()
    .reset_index(name="count")
)
top10 = hotspots.sort_values("count", ascending=False).head(10)

# Visualization: print summaries
print("📊 Shared genes per chromosome:")
print(chr_counts.sort_values("gene_count", ascending=False).head(10))

print("\n🔥 Top 10 hotspots in 10Mb bins:")
print(top10)


📊 Shared genes per chromosome:
   chromosome  gene_count
0          11         653
1           1           0
22          3           0
21          4           0
20          5           0
19          6           0
18          7           0
17          8           0
16          9           0
15         10           0

🔥 Top 10 hotspots in 10Mb bins:
    chromosome  bin_start_10Mb  count
86          11      60000000.0    269
80          11             0.0    112
81          11      10000000.0     66
84          11      40000000.0     57
87          11      70000000.0     53
83          11      30000000.0     43
85          11      50000000.0     35
82          11      20000000.0     18
123         16      30000000.0      0
124         16      40000000.0      0


C:\Users\12039\AppData\Local\Temp\ipykernel_59820\3527432742.py:58: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["chromosome", "bin_start_10Mb"])
